<h4>-----------------------------------------------------------------------------<br>Copyright (c) 2024, Lucid Vision Labs, Inc.</h4>
<h5> THE  SOFTWARE  IS  PROVIDED  "AS IS",  WITHOUT  WARRANTY  OF  ANY  KIND,<br>EXPRESS  OR  IMPLIED,  INCLUDING  BUT  NOT  LIMITED  TO  THE  WARRANTIES<br>OF  MERCHANTABILITY,  FITNESS  FOR  A  PARTICULAR  PURPOSE  AND<br>NONINFRINGEMENT.  IN  NO  EVENT  SHALL  THE  AUTHORS  OR  COPYRIGHT  HOLDERS<br>BE  LIABLE  FOR  ANY  CLAIM,  DAMAGES  OR  OTHER  LIABILITY,  WHETHER  IN  AN<br>ACTION  OF  CONTRACT,  TORT  OR  OTHERWISE,  ARISING  FROM,  OUT  OF  OR  IN<br>CONNECTION  WITH  THE  SOFTWARE  OR  THE  USE  OR  OTHER  DEALINGS  IN <br> THE  SOFTWARE.<br>-----------------------------------------------------------------------------</h5>

In [ ]:
import time
from datetime import datetime

from arena_api.enums import PixelFormat
from arena_api import enums as _enums
from arena_api.__future__.save import Writer
from arena_api.system import system
from arena_api.buffer import BufferFactory

#### Image Processing: Split Channels
>This example introduces splitting channels of an image and saving split images as JPEG images.

In [ ]:
TAB1 = "  "
pixel_format = PixelFormat.BGR8

In [ ]:
"""
This function waits for the user to connect a device before raising
an exception
"""
tries = 0
tries_max = 6
sleep_time_secs = 10
while tries < tries_max:  # Wait for device for 60 seconds
    devices = system.create_device()
    if not devices:
        print(
            f'Try {tries+1} of {tries_max}: waiting for {sleep_time_secs} '
            f'secs for a device to be connected!')
        for sec_count in range(sleep_time_secs):
            time.sleep(1)
            print(f'{sec_count + 1 } seconds passed ',
                  '.' * sec_count, end='\r')
        tries += 1
    else:
        print(f'Created {len(devices)} device(s)\n')
        break
else:
    raise Exception(f'No device found! Please connect a device and run '
                    f'the example again.')

device = system.select_device(devices)
print(f'Device used in the example:\n\t{device}')

In [ ]:
"""
Setup stream values
"""
tl_stream_nodemap = device.tl_stream_nodemap
tl_stream_nodemap['StreamAutoNegotiatePacketSize'].value = True
tl_stream_nodemap['StreamPacketResendEnable'].value = True

In [ ]:
device.start_stream()
buffer = device.get_buffer()

#### Function to save a JPEG image

In [ ]:
def save(buffer, count):
	'''
	demonstrates saving a JPEG image
	(1) converts image to a displayable pixel format
	(2) prepares image parameters
	(3) prepares image writer
	(4) saves image with configuration parameters
	(5) destroys converted image
	'''

	'''
	convert image
	'''
	converted = BufferFactory.convert(buffer, pixel_format)
	print(f"{TAB1}Converted image to {pixel_format.name}")

	'''
	prepare image writer
	'''
	print(f'{TAB1}Prepare Image Writer')
	writer = Writer()
	writer.pattern = 'images/py_imageprocessing_splitchannels/image_'+ str(count)+'.jpg'
	
	writer.save(converted, quality=75, progressive=False, subsampling=_enums.ScJpegSubsamplingList.SC_NO_JPEG_SUBSAMPLING, optimize=False)
	print(f'{TAB1}Image saved {writer.saved_images[-1]}')

	# Destroy converted buffer to avoid memory leaks
	BufferFactory.destroy(converted)

#### Split Channels

In [ ]:
buffers = BufferFactory.split_channels(buffer)

for count, buf in enumerate(buffers):
	save(buf, count)

In [ ]:
device.requeue_buffer(buffer)

#### Clean up

In [ ]:
device.stop_stream()

# Destroy Device
system.destroy_device()